In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.db as db

import pandas as pd
import pathlib
import sqlalchemy as sa

import numpy as np

## Applies debarcoding

In [ ]:
C = sm.input

In [ ]:
bro = spb.get_bro(C.fn_config, readonly=False)

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())


print(n_img, n_obj)

In [ ]:
%matplotlib inline

In [ ]:
%%time
p = bro.processing.debarcode.plot_histograms(dist=30,
                                             stack='FullStackFiltered',
                                             measurement_name='MeanIntensityComp',
                                            transform='np.log10(value+0.01)')

print(p)

In [ ]:
%%time
bro.processing.debarcode.debarcode(dist=30,
                                   stack='FullStackFiltered',
                                   measurement_name='MeanIntensityComp',
                                   #bc_treshs=manual_co
                                   transform=lambda x: np.log10(x+0.01)
                                  )

In [ ]:
%matplotlib inline

In [ ]:
%%time
bro.plots.debarcoedequality.quality_plot()

This will delete not (clearly) debarcoded spheres

In [ ]:
q_debar=(bro.data.main_session.query(db.images.image_id
                                )
     .filter(sa.and_(db.images.bc_highest_count > 10,db.images.bc_highest_count/(db.images.bc_second_count+1) > 2 ))
    ).subquery()
stmt = bro.session.query(db.valid_images).filter(sa.not_(db.valid_images.image_id.in_(q_debar)))

In [ ]:
print(f'{stmt.count()} sphere sections not unambiguously debarcoded')

In [ ]:
stmt.delete(synchronize_session='fetch')
bro.session.commit()

In [ ]:
n_cond = (bro.session.query(db.conditions.condition_id)
            .join(db.images)
             .join(db.valid_images)
              .group_by(db.conditions.condition_id).count())
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())




print(n_cond,n_img, n_obj)

This plots 10 random spheres

In [ ]:
import matplotlib.pyplot as plt
import random
import colorcet
from matplotlib_scalebar.scalebar import ScaleBar

def add_scalebar(
    ax, resolution=0.000001, location=4, color="white", pad=0.5, frameon=False, fixed_value=None, **kwargs
):
    scalebar = ScaleBar(
        resolution, location=location, color=color, pad=pad, frameon=frameon,fixed_value=fixed_value, **kwargs
    )  # 1 pixel = 0.2 meter
    ax.add_artist(scalebar)

n=10
ids = [c[0] for c in  bro.session.query(db.valid_images.image_id).all()]

random.seed(234)
random.shuffle(ids)

fig, axs = plt.subplots(ncols=n)
for i, (imid, ax) in enumerate(zip(ids, axs)):
    has_cb = i == (n-1)
    bro.plots.debarcoededcells.plot_debarcoded_cells(imid, ax=ax, colorbar=has_cb,
                                                     base_colormap=colorcet.glasbey_dark,
                                                     color_invalid='white',
                                                     cmap_mask='Greys',
                                                     cmap_mask_alpha=0.6
                                                )
    add_scalebar(ax, fixed_value=0.00005, label_formatter=lambda value, unit: '',
                height_fraction=0.05)
fig.set_figwidth(30)

In [ ]:
fig.savefig('./results/figures/subfig_example_debarcoded.pdf', dpi=800)

In [ ]:
fig.savefig('./results/figures/subfig_example_debarcoded.svg', dpi=800)